In [1]:
import numpy as np
import pandas as pd
import math
import pickle
import json
from numpy.random import rand, randint, exponential, normal, uniform, lognormal, dirichlet

In [2]:
data_dir = './data/'

In [3]:
# functions for dumping and loading generated data files
def pickle_dump(file_to_dump, file_dir):
  with open(file_dir, "wb") as fp:   #Pickling
    pickle.dump(file_to_dump, fp)

def pickle_load(file_dir):
  with open(file_dir, "rb") as fp:   # Unpickling
    return pickle.load(fp)

def json_dump(file_to_dump, file_dir):
    with open(file_dir, 'w') as fp:
        json.dump(file_to_dump, fp)

def json_load(file_dir):
    with open(file_dir) as f:
        return json.load(f)

### <b>1.Sets initialization</b>

B: set of bike-stations, B = {1, ..., B} <br>
S: set of scenarios, S = {1,...,S} or finit set of possible realization of the uncertainty.

In [4]:
# Sets
Bset = 'A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V'.split(',')  # 22 Bike Stations
Bij_set = [(bike2, bike) for bike2 in Bset for bike in Bset]     # Bike pairs 
scenario_num = 100                                               # Total scenario numbers (using for now)
Sset = ['S'+str(s) for s in range(scenario_num)]                 # Scenario sets

In [6]:
# dump sets
pickle_dump(Bset, data_dir + 'Bset.txt')
pickle_dump(Bij_set, data_dir + 'Bij_set.txt')
pickle_dump(Sset, data_dir+'Sset_' + str(scenario_num) + '.txt')

#### <b>2.Deterministic parameters</b>

c: procurement cost per bike at each bike-station at the beginning of the service <br>
$v_i$: stock-out cost per bike at bike-station i $\in$ B <br>
$w_i$: time-waste cost per bike due to overflow at bike-station i $\in$ B <br>
$t_{ij}$: unit transshipment cost per bike transshiped from bike-station i to bike-station j, i,j $
\in$ B <br>
$k_i$: capacity of bike-station i $\in$ B <br>
$\xi_{ijk}$: rental demand from bike-station i to bike-station j in scenario s, i, j $\in$ B, s $\in$ S  <br>
$p_s \in [0,1]$: probability of scenario s $\in$ S, $\sum_{s=1}^{S} p_s$ = 1<br>



In [15]:
np.random.seed(1612)

In [ ]:
c = 2         # unit procurement cost
v_i = 4       # stock-out cost
w_i = 8       # time-waste cost 
t_ij = 1      # unit transshipment cost 
deterministic_params = {'c': c, 'v_i': v_i, 'w_i': w_i, 't_ij': t_ij}

# capacity for each station 
k_i = [22, 10, 10, 17, 19, 12, 20, 8, 8, 20, 10, 19, 8, 10, 10, 10, 18, 10, 8, 12, 12, 10]
capacity_i = {Bset[index] : k_i[index] for index in range(len(Bset))}
capacity_i_constant = {station: 30 for station in Bset}   # for the constant capacity case (not using for now)

# Stochastic demand
np.random.seed(1612)
demScens = {}
prob = np.around(np.random.dirichlet(np.ones(len(Sset))), decimals = 7)      # not consistent?
p_s = {scenario: prob[int(''.join(list(scenario)[1:]))]  for scenario in Sset}

# Generate stochastic demand for each station pair 
distribution_choice = 'log-normal'

# Generate demand using log-normal distribution 
if distribution_choice == 'log-normal':
  # get log-normal distribution information 
  log_normal_mean = {station_pair: randint(math.log(2), math.log(6)) for station_pair in Bij_set}   #based on table 12 result /22, shifted +2
  log_normal_stdev = {station_pair: rand() for station_pair in Bij_set}

  # stochastic demand stored in form {((B_i, B_j), S_s) : demand}
  for station_pair in Bij_set:
    for scenario in Sset:
      # diagonal demands set to 0 
      if station_pair[0] == station_pair[1]:
        demScens[(station_pair[0], station_pair[1], scenario)] = 0
      else:
        demScens[(station_pair[0], station_pair[1], scenario)] = int(lognormal(log_normal_mean[station_pair], log_normal_stdev[station_pair]))


In [24]:
# dump data
json_dump(deterministic_params, data_dir + 'deterministic_params.json')
json_dump(capacity_i, data_dir + 'capacity_i.json')
json_dump(capacity_i_constant, data_dir + 'capacity_i_constant.json')
json_dump(p_s, data_dir + 'p_s_' + str(len(p_s)) + '.json')
pickle_dump(demScens, data_dir + 'demScens_' + distribution_choice + '_' + str(len(Sset)) + '.dict')

In [6]:

stdev_ij = {station_pair: rand() for station_pair in Bij_set}

{('A', 'A'): 0.4680944820615274,
 ('A', 'B'): 0.06507165380237634,
 ('A', 'C'): 0.18868055389416738,
 ('A', 'D'): 0.8481538252550054,
 ('A', 'E'): 0.8654458178191857,
 ('A', 'F'): 0.3245304625094664,
 ('A', 'G'): 0.0002443192954681983,
 ('A', 'H'): 0.4049127138172093,
 ('A', 'I'): 0.05124483320581308,
 ('A', 'J'): 0.5240286971368742,
 ('A', 'K'): 0.19658537256667452,
 ('A', 'L'): 0.11347240944541404,
 ('A', 'M'): 0.10197529554397455,
 ('A', 'N'): 0.026019253305078593,
 ('A', 'O'): 0.5024811514706562,
 ('A', 'P'): 0.2546866147028911,
 ('A', 'Q'): 0.240531223438179,
 ('A', 'R'): 0.9789247032404399,
 ('A', 'S'): 0.8055606820693102,
 ('A', 'T'): 0.754591219192524,
 ('A', 'U'): 0.9996123955347371,
 ('A', 'V'): 0.5362433163018974,
 ('B', 'A'): 0.7421435534677737,
 ('B', 'B'): 0.5071543102754207,
 ('B', 'C'): 0.14462049592874082,
 ('B', 'D'): 0.8850055802195385,
 ('B', 'E'): 0.6776488787784871,
 ('B', 'F'): 0.620034830898729,
 ('B', 'G'): 0.44305580621593565,
 ('B', 'H'): 0.54193067994084,
 (